# Packages


In [87]:
import spacy
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import lyricsgenius
from tqdm import tqdm
import json
import duden
import time
from collections import Counter
import pandas as pd
import csv


# Get Lyrics


- API Schlüssel und Zugangsdaten einsetzen
- Playlist IDs von Spotify suchen 
- Jeder Track der Playlist wird auf LyricsGenius gesucht
- Sollte zu einem Song kein Lyrics gefunden werden, wird die Anzahl hochgezählt
- Es wird eine JSON Datei gebildet

In [6]:
SPOTIPY_CLIENT_ID = '46da6046625f409cb53ab06bf807861d'
SPOTIPY_CLIENT_SECRET = '00efb0ab68db4f1eaf27ec53a4c1e068'
GENIUS_ACCESS_TOKEN = 'OsBvEsJ_OLKYR42CJoaA_N97GNOS44XXFk4KPPcLtCYnWmDwnnzY2RpcG_SdG25a'

Ausführen von getlyrics.py

## Analysis

In [10]:
with open("./all_songs_neu2.json") as f:
    songs_dict = json.load(f)

encoded= json.dumps (songs_dict, ensure_ascii=True)
songs_dict_0 =json.loads (encoded)

1. Spacy

Ausführen von cyspa.py

Hier werden alle Nomen ausgegeben in einer csv datei: SongID, Nomen, Geschlecht, Release Year, Artist

In [11]:
#NEU #Spacy muss gedownloaded werden mit : pip install -U pip setuptools wheel   und    pip install spacy 
#de_core_news_sm-3.5.0-py3-none-any.whl (muss gedownloaded werden -> https://spacy.io/models/de/)
#de_core_news_lg-3.5.0-py3-none-any.whl
#de_dep_news_trf-3.5.0-py3-none-any.whl


nlp2 = spacy.load ("de_core_news_lg")
playlist="Spotify"
song_id=0

with open ('Nomen_csv.csv','w', newline ='') as nomen_file:
    writer = csv.writer(nomen_file, delimiter=';', lineterminator='\n')
    writer.writerow (["ID","Noun", "Gender", "Year", "Artist"])    

    for song_id in tqdm(range(len(songs_dict_0[playlist]))):

        lyrics_from_song = songs_dict_0[playlist][str(song_id)]["lyrics"] #hier wird aus dem der Datei mit allen Songtexten nur das Lyrics abgefragt
        year_from_song = songs_dict_0[playlist][str(song_id)]["release_year"]
        artist_song=songs_dict_0[playlist][str(song_id)]["artist_name"]
        doc3 = nlp2 (lyrics_from_song)

        for token in doc3: 
            if token.pos_ == "NOUN":
                    try:
                        row= ([song_id, token.text,token.morph.get('Gender'), year_from_song, artist_song])
                        writer.writerow(row)
                    except:
                        print(f"Error at Song {song_id} with word {token.text}.")
                        pass
            else:
                pass


 11%|█▏        | 10/88 [00:00<00:04, 16.93it/s]

Error at Song 6 with word jеdem.
Error at Song 6 with word wеrden.


 42%|████▏     | 37/88 [00:01<00:02, 21.41it/s]

Error at Song 34 with word siе.


 68%|██████▊   | 60/88 [00:02<00:01, 24.39it/s]

...

 82%|████████▏ | 72/88 [00:03<00:00, 20.83it/s]

Error at Song 67 with word Совета.
Error at Song 67 with word против.


 92%|█████████▏| 81/88 [00:03<00:00, 21.70it/s]

Error at Song 78 with word hеute.


100%|██████████| 88/88 [00:04<00:00, 20.78it/s]


TODO :
- Gleiche Wortstämme zusammenfassen

NOMEN

In [24]:
scrape_data = False # decide, whether to scrape full data again or read an exisiting json file.

### Mit Genderliste vergleichen

TODO:
- mit Genderliste Vergleichen
- mit Ausdrucks-Worliste vergleichen
- mit Frau/Mann List vergleichen

In [96]:
### Abfrage Genderneutrale Nomen:
import pandas as pd

complete_df = pd.DataFrame(columns=['Noun', 'Year', 'Artist'])

gender_neu = pd.read_csv("./gender_neu_Nomen.csv",sep=";",header=None,encoding="utf8")
gender_neu = gender_neu.iloc[:, 0]
gender_neu.columns = ["Wort"]

df = pd.read_csv(r"Nomen_csv.csv", encoding='ISO-8859-1', sep=';')
nomen= df.iloc[:,1]
id=0
id_1=0


for idx, wort in df.iterrows():
    for gender in list(gender_neu):
        # print(wort, gender)
        if wort["Noun"] == gender:
            complete_df.loc[len(complete_df)] = [wort["Noun"], wort["Year"], wort["Artist"]]
            




""" if id_1 < len(gender_neu)-1:
    if nomen[id] == gender_neu[id_1]:
        print(True)
    else:
        id_1+=1 
        
else:
    print(False)
    id +=1 """


' if id_1 < len(gender_neu)-1:\n    if nomen[id] == gender_neu[id_1]:\n        print(True)\n    else:\n        id_1+=1 \n        \nelse:\n    print(False)\n    id +=1 '

In [97]:
complete_df.head()

,Noun,Year,Artist
0,Nachbarschaft,2022,Die Toten Hosen
1,Kind,2022,Die Toten Hosen
2,Kind,2022,Die Toten Hosen
3,Kind,2022,Die Toten Hosen
4,Sonnenschein,2022,Die Toten Hosen


In [98]:
complete_df.to_csv("./complete_df.csv")

In [83]:
df.columns

Index(['ID', 'Noun', 'Gender', 'Year', 'Artist'], dtype='object')

In [85]:
for idx, row in df.iterrows():
    print(row["Noun"])
    break


stern


In [ ]:
df= pd.read_csv(r"Nomen_csv.csv", encoding='ISO-8859-1', sep=';')
df.sort_values(by='Noun')
df.iloc[:,1]

In [30]:
df.iloc[:,3].keys()

RangeIndex(start=0, stop=3439, step=1)

In [39]:
###noch nicht funktionstüchtig
complete_df = pd.DataFrame(columns=["Wort","neutral", "Jahr"])
tmp = []
playlist='Prototyp'
for id in df.iloc[:,0].keys():
    for g in df.iloc[:,2].keys(): # geschlecht
        w = df.iloc[:,1] # wort
        is_gender_neutral = True if w in gender_neu else False

        complete_df.loc[len(complete_df)] = [g, w, is_gender_neutral]
        tmp.append(g)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().